# Automatic speech transcription using Whisper
In this module, we use [Whisper](https://github.com/openai/whisper) from OpenAI to transcribe speech automatically. Whisper is a robust automatic speech recognition (ASR) model that supports about 100 different languages (e.g., English, Italian, Dutch, Japanese, Chinese, Spanish, etc).

Whisper provides 5 multilingual model sizes as follows:

|  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
|  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
|  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
| small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
| medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
| large  |   1550 M   |        N/A         | `large/large-v2/large-v3` |    ~10 GB     |       1x       |

As you can see, the smaller the model is, the faster computational time is, with less accurate results.


### Issues in timestamps accuracy
The original whisper models do not correctly capture silences. Rather, they just include sliences in timestamps. <br>
For example, if an utterance was produced from 00:00:11.000 to 00:00:15.000 followed by a 2-second silence, the timestamp will be 00:00:11.000 - 00:00:17.000 instead. <br>

Also, the original whisper model doesn't provide word-level timestamps.

To solve this, we will use **[whisperx](https://github.com/m-bain/whisperX)**, which first transcribes speech using faster-whisper model and use a forced-alignment algorithm for improved timing accuracy and word-level timestamps.


***

## Overview of the script

The script performs following processes in the listed order:

1. Import packages and define path for the input and output folders
1. Load a whisper model
1. Go through each audio/video (wav/mp4) files in the "input" folder and run whisper on each file
    1. Transcribe audio/video file using faster-whisper
    1. Get accurate timestamps using a forced alignment algorithm
    1. Move on to the next file
1. Call the "export_transcript_as_tsv()" function to export the whisper output as a tab-delimited text file

### Import packages and define paths

### Import packages and define path for input and output folders
Let's first import required packages.


<details><summary><font color = "mandarin">If you haven't installed the packages, click here and follow the steps below</font></summary>

1. Open terminal/anaconda prompt at the folder in which you store this notebook
    - Mac:
        1. go to the "imprs_whisper_workshop" folder
        1. right-click the "imprs_whisper_workshop" folder
        1. click "open terminal at this folder" <br><br>
    - Windows:
        1. go to the "imprs_whisper_workshop" folder
        1. copy the path to the folder
        1. open Anaconda Prompt
        1. type cd and paste the path after a space (e.g., cd D:/users/shoakamine/imprs_whisper_workshop)
        1. if the folder is not in the C drive, type the drive initial followed by a colon (e.g., D:):

1. Create Python3.10 environment
    ```
    conda create --name whisperx python=3.10
    ```
1. Activate your conda environment
    ```
    conda activate whisperx
    ```

1. Install PyTorch following the instructions [here](https://pytorch.org/get-started/previous-versions/#v200)

1. Run this command to install required packages: `pip install -r requirements.txt`

1. Click "select kernel" on the top right of VS code and select "whisperx"


In [2]:
import whisperx
import pandas as pd
import os
import torch

# import custom-made functions
from tsv_export import export_transcript_as_tsv, export_transcript_as_tsv_textonly


input_folder = "../input/"
output_folder = "../output/"

objc[28654]: Class AVFFrameReceiver is implemented in both /usr/local/Cellar/ffmpeg/6.1.1_6/lib/libavdevice.60.3.100.dylib (0x10e510378) and /Users/shoakamine/opt/anaconda3/envs/whisperx/lib/python3.10/site-packages/av/.dylibs/libavdevice.60.1.100.dylib (0x1383870f8). One of the two will be used. Which one is undefined.
objc[28654]: Class AVFAudioReceiver is implemented in both /usr/local/Cellar/ffmpeg/6.1.1_6/lib/libavdevice.60.3.100.dylib (0x10e5103c8) and /Users/shoakamine/opt/anaconda3/envs/whisperx/lib/python3.10/site-packages/av/.dylibs/libavdevice.60.1.100.dylib (0x138387148). One of the two will be used. Which one is undefined.
/Users/shoakamine/opt/anaconda3/envs/whisperx/lib/python3.10/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


### Run the model and export the output as tsv

<details><summary>Click here for the list of languages</summary>

1.  "en": "english",
1.  "zh": "chinese",
1. "de": "german",
1. "es": "spanish",
1. "ru": "russian",
1. "ko": "korean",
1. "fr": "french",
1. "ja": "japanese",
1. "pt": "portuguese",
1. "tr": "turkish",
1. "pl": "polish",
1. "ca": "catalan",
1. "nl": "dutch",
1. "ar": "arabic",
1. "sv": "swedish",
1. "it": "italian",
1. "id": "indonesian",
1. "hi": "hindi",
1. "fi": "finnish",
1. "vi": "vietnamese",
1. "he": "hebrew",
1. "uk": "ukrainian",
1. "el": "greek",
1. "ms": "malay",
1. "cs": "czech",
1. "ro": "romanian",
1. "da": "danish",
1. "hu": "hungarian",
1. "ta": "tamil",
1. "no": "norwegian",
1. "th": "thai",
1. "ur": "urdu",
1. "hr": "croatian",
1. "bg": "bulgarian",
1. "lt": "lithuanian",
1. "la": "latin",
1. "mi": "maori",
1. "ml": "malayalam",
1. "cy": "welsh",
1. "sk": "slovak",
1. "te": "telugu",
1. "fa": "persian",
1. "lv": "latvian",
1. "bn": "bengali",
1. "sr": "serbian",
1. "az": "azerbaijani",
1. "sl": "slovenian",
1. "kn": "kannada",
1. "et": "estonian",
1. "mk": "macedonian",
1. "br": "breton",
1. "eu": "basque",
1. "is": "icelandic",
1. "hy": "armenian",
1. "ne": "nepali",
1. "mn": "mongolian",
1. "bs": "bosnian",
1. "kk": "kazakh",
1. "sq": "albanian",
1. "sw": "swahili",
1. "gl": "galician",
1. "mr": "marathi",
1. "pa": "punjabi",
1. "si": "sinhala",
1. "km": "khmer",
1. "sn": "shona",
1. "yo": "yoruba",
1. "so": "somali",
1. "af": "afrikaans",
1. "oc": "occitan",
1. "ka": "georgian",
1. "be": "belarusian",
1. "tg": "tajik",
1. "sd": "sindhi",
1. "gu": "gujarati",
1. "am": "amharic",
1. "yi": "yiddish",
1. "lo": "lao",
1. "uz": "uzbek",
1. "fo": "faroese",
1. "ht": "haitian creole",
1. "ps": "pashto",
1. "tk": "turkmen",
1. "nn": "nynorsk",
1. "mt": "maltese",
1. "sa": "sanskrit",
1. "lb": "luxembourgish",
1. "my": "myanmar",
1. "bo": "tibetan",
1. "tl": "tagalog",
1. "mg": "malagasy",
1. "as": "assamese",
1. "tt": "tatar",
1. "haw": "hawaiian",
1. "ln": "lingala",
1. "ha": "hausa",
1. "ba": "bashkir",
1. "jw": "javanese",
1. "su": "sundanese"

In [3]:
### Load the whisper model
# set the device, batch size, and compute type
device = 'cuda' if torch.cuda.is_available() else 'cpu' # setting device on GPU if available, else CPU
batch_size = 16 if device == "cuda" else 4 # reduce to 4 if low on GPU memory
compute_type = "float16" if device == "cuda" else "default"
model_size = "base" # change to "large-v3" for better performance (note that it is significantly slower)
print(f"* Using device: {device} \n* Batch size: {batch_size} \n* Model size: {model_size} \n* Compute type: {compute_type}")

# load model from whisper
model = whisperx.load_model(model_size, device, compute_type=compute_type)


### Transcribe all audio/video files in the input folder
# iterate over files in the videos folder & apply whisper model on each videos
for filename in os.listdir(input_folder):
    path = os.path.join(input_folder, filename)

    # check if it is a wav file
    if filename.endswith(".wav") or filename.endswith(".mp4"):
        # check if the output file already exists
        output_filename = filename.split(".")[0] + ".txt"
        if os.path.exists(os.path.join(output_folder, output_filename)):
            print(f"{output_filename} already exists in the output folder")
        else:
            #apply whisper model on each file
            print(">>>>>> Now, whisper is working on " + filename)

            # 1. Transcribe with original whisper (batched)
            audio = whisperx.load_audio(path)
            result = model.transcribe(audio, batch_size=batch_size)
            print(">>>>>> Transcription complete. Now aligning the text with the audio...")

            # 2. Align whisper output
            model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
            result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

            #export the results as tsv
            # export_transcript_as_tsv(result, output_filename, output_folder)
            export_transcript_as_tsv_textonly(result, output_filename, output_folder)




* Using device: cpu 
* Batch size: 4 
* Model size: base 
* Compute type: default


[2024-04-12 19:30:59.304] [ctranslate2] [thread 687705] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


No language specified, language will be first be detected for each audio file (increases inference time).


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../../../.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.2.1. Bad things might happen unless you revert torch to 1.x.
>>>>>> Now, whisper is working on salma_hayek_short.mp4


: 

### <font color="orange">Exercise 1: Import transcript to ELAN</font>
Let's import the transcript to ELAN. [Here](https://www.mpi.nl/corpus/html/elan/ch04s03s01.html#Sec_Importing_CSV_Tab-delimited_Text_Files)'s official documentation of ELAN for importing csv files.

### <font color="orange">Exercise 2: Change the model size</font>
Change the model_size in the code above to "large-v2" and run the Whisper model again. After running the model, answer the following questions:

- Is the output more accurate compared to the based model?
- How long did it take for the large-v2 model to process a 37 seconds video?
- Did audience's voice affect the transcription accuracy?

*Make sure to add "_base" to the filename in the output folder. This is because whisper model won't run if there's a csv file with the same filename as the input wav file.